Import

In [1]:
import sys

sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import re
import cv2
import json
from pprint import pprint
import random

Helper Functions

In [2]:
def extract_id(file_name: str) -> str | None:
    """
    Give a file name such as 'A_P000001_PAS_CPG.tif',
    Extract the file ID: 'A_P000001' using regular expression
    """
    match = re.match(r"([A-Z]_P\d+)_", file_name, re.IGNORECASE)

    if match:
        return match.group(1)
    else:
        return None


def cell_mask_to_rgb(cell_mask: np.ndarray) -> np.ndarray:
    """For visualization purposes
    Convert 2D cell mask to an RGB image
    2D cell mask:
    1 - Lympchoyte centroids
    2 - Monocytes centroids
    RGB image:
    Green - Lympchoyte centroids
    Blue - Monocytes centroids
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    cell_mask = cv2.dilate(cell_mask, kernel, iterations=1)
    rgb_mask = np.zeros(
        (cell_mask.shape[0], cell_mask.shape[1], 3), dtype=np.uint8
    )
    rgb_mask[cell_mask == 1] = (0, 255, 0)
    rgb_mask[cell_mask == 2] = (0, 0, 255)
    return rgb_mask

In [3]:
def colour_transformation(image: np.ndarray) -> np.ndarray:
    # image = cv2.GaussianBlur(image, (3, 3), 0)
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    b_channel = lab[:, :, 2] * -1
    normalised_map = (b_channel - np.amin(b_channel)) / (
        np.amax(b_channel) - np.amin(b_channel)
    )

    # ret, _ = cv2.threshold(
    #     b_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
    # )
    # blue_map = b_channel - ret
    # blue_map[blue_map < 0] = 0
    # blue_map = blue_map * lab[:, :, 0]

    fig, axes = plt.subplots(1, 3, figsize=(10, 5))
    axes[0].imshow(lab[:, :, 0], cmap="jet")
    axes[0].set_title("L")
    axes[1].imshow(lab[:, :, 1], cmap="jet")
    axes[1].set_title("A")
    axes[2].imshow(normalised_map, cmap="jet")
    axes[2].set_title("B")
    for ax in axes:
        ax.axis("off")
    plt.show()

Visualize 5 random patches

In [ ]:
# Specify data folders

# Folder containing image patches
patch_image_dir = "/home/u1910100/Documents/Monkey/patches_512/images"
# Folder containing cell masks
cell_mask_dir = (
    "/home/u1910100/Documents/Monkey/patches_512/annotations/masks"
)
# Folder containing json files
json_dir = (
    "/home/u1910100/Documents/Monkey/patches_512/annotations/json"
)


# Visualize 50 random patches
file_names = os.listdir(patch_image_dir)
files_sample = random.sample(file_names, 1)

for name in files_sample:
    name_without_ext = os.path.splitext(name)[0]

    # Get path to files
    patch_image_path = os.path.join(
        patch_image_dir, f"{name_without_ext}.npy"
    )
    cell_mask_path = os.path.join(
        cell_mask_dir, f"{name_without_ext}.npy"
    )
    json_path = os.path.join(json_dir, f"{name_without_ext}.json")

    # load RGB patch and cell mask
    image_patch = np.load(patch_image_path)
    colour_transformation(image_patch)
    cell_mask = np.load(cell_mask_path)
    # Convert cell mask to RGB image for visualization
    rgb_cell_mask = cell_mask_to_rgb(cell_mask)

    # load json file
    annotations = {}
    with open(json_path, "r") as file:
        annotations = json.load(file)

    # Output
    pprint(name)
    # pprint(annotations)
    fig, axes = plt.subplots(1, 3, figsize=(17, 17))
    axes[0].imshow(image_patch)
    axes[0].title.set_text("RGB Patch")
    axes[1].imshow(rgb_cell_mask)
    axes[1].title.set_text("Cell Mask")
    axes[2].imshow(image_patch, alpha=0.5)
    axes[2].imshow(rgb_cell_mask, alpha=0.5)
    axes[2].title.set_text("Cell Mask overlay on RGB Patch")
    plt.show()

Count cells

In [ ]:
# Folder containing json files
json_dir = (
    "/home/u1910100/Documents/Monkey/patches_512/annotations/json"
)
file_names = os.listdir(json_dir)

lymphocyte_count = 0
monocytes_count = 0

for name in file_names:
    json_path = os.path.join(json_dir, name)
    annotations = {}
    with open(json_path, "r") as file:
        annotations = json.load(file)

    lymph_coords = annotations["lymphocytes"]
    mono_coords = annotations["monocytes"]

    lymphocyte_count += len(lymph_coords)
    monocytes_count += len(mono_coords)


print(f"lympho count = {lymphocyte_count}")
print(f"mono count = {monocytes_count}")

In [ ]:
# Create patch stats
json_dir = (
    "/home/u1910100/Documents/Monkey/patches_512/annotations/json"
)

stats_save_path = (
    "/home/u1910100/Documents/Monkey/patches_512/patch_stats.json"
)

patch_stats = {}

p = 0
n = 0

file_names = os.listdir(json_dir)
for name in file_names:
    name_without_ext = os.path.splitext(name)[0]
    json_path = os.path.join(json_dir, name)
    annotations = {}
    with open(json_path, "r") as file:
        annotations = json.load(file)

    lymph_coords = annotations["lymphocytes"]
    mono_coords = annotations["monocytes"]

    patch_stats[name_without_ext] = {
        "lymph_count": len(lymph_coords),
        "mono_count": len(mono_coords),
    }

    if len(lymph_coords) > 0 or len(mono_coords) > 0:
        p += 1
    else:
        n += 1

print(p)
print(n)
with open(stats_save_path, "w") as file:
    json.dump(patch_stats, file)

Visualize Detection Dataloader

In [ ]:
%reset -f
import sys

sys.path.append("../")
from monkey.data.dataset import get_detection_dataloaders
from monkey.data.data_utils import imagenet_denormalise
from monkey.config import TrainingIOConfig
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

use_nuclick_masks = True
batch_size = 8
module = "detection"

IOconfig = TrainingIOConfig(
    dataset_dir="/Users/wildsquirrel/Documents/Monkey/patches_256",
    save_dir="./",
)
if use_nuclick_masks:
    IOconfig.set_mask_dir(
        "/Users/wildsquirrel/Documents/Monkey/patches_256/annotations/nuclick_masks_processed"
    )


train_loader, val_loader = get_detection_dataloaders(
    IOconfig,
    val_fold=1,
    dataset_name="detection",
    batch_size=batch_size,
    do_augmentation=True,
    module=module,
    target_cell_type="mono",
)

In [2]:
def binary_to_rgb(binary_mask):
    rgb = np.zeros(
        (binary_mask.shape[0], binary_mask.shape[1], 3),
        dtype=np.uint8,
    )
    rgb[:, :, 0] = binary_mask * 255
    rgb[:, :, 1] = binary_mask * 255
    rgb[:, :, 2] = binary_mask * 255
    return rgb

In [ ]:
data = next(iter(train_loader))
print(module)
print(data["id"])
fig, axes = plt.subplots(batch_size, 3, figsize=(12, 18))

for i in range(batch_size):
    image = data["image"][i].numpy()
    image = np.moveaxis(image, 0, 2)
    image = imagenet_denormalise(image)
    axes[i][0].imshow(image)

    if module == "detection":
        inflamm_mask = data["mask"][i][0].numpy()
        axes[i][1].imshow(inflamm_mask, cmap="gray")
        axes[i][2].imshow(image, alpha=0.5)
        axes[i][2].imshow(inflamm_mask, alpha=0.5)

    if module == "multiclass_detection":

        inflamm_mask = data["mask"][i].numpy()
        axes[i][1].imshow(inflamm_mask[0], cmap="jet")
        axes[i][1].set_title("lymphocytes")
        axes[i][2].imshow(inflamm_mask[1], cmap="jet")
        axes[i][2].set_title("monocytes")

        for ax in axes.ravel():
            ax.axis("off")

plt.show()

Visualize Classification DataLoader

In [ ]:
%reset -f
import sys

sys.path.append("../")
from monkey.data.dataset import get_classification_dataloaders
from monkey.data.data_utils import imagenet_denormalise
from monkey.config import TrainingIOConfig
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import os

IOconfig = TrainingIOConfig(
    dataset_dir="/home/u1910100/Documents/Monkey/classification",
    save_dir="./",
)
IOconfig.set_image_dir(
    "/home/u1910100/Documents/Monkey/classification/patches"
)
IOconfig.set_mask_dir(
    "/home/u1910100/Documents/Monkey/classification/patches"
)
batch_size = 2
train_loader, val_loader = get_classification_dataloaders(
    IOconfig,
    val_fold=1,
    batch_size=batch_size,
    do_augmentation=True,
)

In [ ]:
data = next(iter(val_loader))
fig, axes = plt.subplots(batch_size, 2, figsize=(8, 8))

for i in range(batch_size):
    print(data["id"][i])
    print(data["label"][i])
    # print(data["label"].shape)
    # break
    image = data["image"][i].numpy()
    inflamm_mask = data["mask"][i][0].numpy()

    image = np.moveaxis(image, 0, 2)
    image = imagenet_denormalise(image)

    axes[i][0].imshow(image)
    axes[i][1].imshow(inflamm_mask, cmap="gray")
plt.show()

Visualize Multitask Dataset

In [ ]:
%reset -f
import sys

sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import os
import random
from monkey.data.dataset import get_detection_dataloaders
from monkey.config import TrainingIOConfig
from monkey.data.data_utils import imagenet_denormalise


IOconfig = TrainingIOConfig(
    dataset_dir="/media/u1910100/data/Monkey/patches_256",
    save_dir="./",
)

IOconfig.set_mask_dir(
    "/media/u1910100/data/Monkey/patches_256/annotations/nuclick_masks_processed_v2"
)

batch_size = 8
train_loader, val_loader = get_detection_dataloaders(
    IOconfig,
    val_fold=1,
    dataset_name="multitask",
    batch_size=batch_size,
    do_augmentation=True,
    disk_radius=11,
    augmentation_prob=0.95,
    strong_augmentation=True,
)

# def binary_to_color(mask):
#     rgb = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
#     rgb[:, :, 0] = np.where(mask == 1, 255, 0)
#     rgb[:, :, 1] = np.where(mask == 2, 255, 0)
#     return rgb

In [ ]:
batch_size = 1
train_loader, val_loader = get_detection_dataloaders(
    IOconfig,
    val_fold=1,
    dataset_name="multitask",
    batch_size=batch_size,
    do_augmentation=True,
    disk_radius=11,
    augmentation_prob=0.95,
    strong_augmentation=True,
)

data = next(iter(train_loader))
image = data["image"][0].numpy()
image = np.moveaxis(image, 0, 2)
image = imagenet_denormalise(image)
inflamm_mask = data["inflamm_mask"][0][0].numpy()

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(image)
axes[0].title.set_text("RGB Patch")
axes[1].imshow(inflamm_mask, cmap="gray")
axes[1].title.set_text("Inflammation Mask")

from scipy import ndimage
import cv2
def get_gradient_maps(image, mask):

    if np.sum(mask) == 0:
        return np.zeros_like(image), np.zeros_like(image)
    
    # convert image to gray scale
    image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140])

    sx = ndimage.sobel(image, axis=0)
    sy = ndimage.sobel(image, axis=1)
    # normalize to [-1, 1]
    sx = sx / np.max(np.abs(sx))
    sy = sy / np.max(np.abs(sy))
    sx[mask == 0] = 0
    sy[mask == 0] = 0

    # Apply Gaussian filter
    sx = cv2.GaussianBlur(sx, (3, 3), 0)
    sy = cv2.GaussianBlur(sy, (3, 3), 0)
    return sx, sy

sx, sy = get_gradient_maps(image, inflamm_mask)
print(sx.shape)
print(np.min(sx), np.max(sx))
print(np.min(sy), np.max(sy))
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(sx, cmap="jet")
axes[0].title.set_text("Sobel X")
axes[1].imshow(sy, cmap="jet")
axes[1].title.set_text("Sobel Y")


In [ ]:
data = next(iter(train_loader))

print(data["id"])
fig, axes = plt.subplots(batch_size, 13, figsize=(30, 40))

for i in range(batch_size):
    image = data["image"][i].numpy()
    image = np.moveaxis(image, 0, 2)
    image = imagenet_denormalise(image)
    axes[i][0].imshow(image)

    inflamm_mask = data["inflamm_mask"][i].numpy()
    axes[i][1].imshow(inflamm_mask[0], cmap="gray")
    # axes[i][1].imshow(image, alpha=0.3)
    axes[i][1].set_title("inflamm mask")

    lymph_mask = data["lymph_mask"][i].numpy()
    axes[i][2].imshow(lymph_mask[0], cmap="gray")
    # axes[i][2].imshow(image, alpha=0.3)
    axes[i][2].set_title("lymph")

    mono_mask = data["mono_mask"][i].numpy()
    axes[i][3].imshow(mono_mask[0], cmap="gray")
    # axes[i][3].imshow(image, alpha=0.3)
    axes[i][3].set_title("mono")

    inflamm_contour_mask = data["inflamm_contour_mask"][i].numpy()
    axes[i][4].imshow(inflamm_contour_mask[0], cmap="gray")
    axes[i][4].set_title("contour")

    lymph_contour_mask = data["lymph_contour_mask"][i].numpy()
    axes[i][5].imshow(lymph_contour_mask[0], cmap="gray")
    axes[i][5].set_title("lymph contour")

    mono_contour_mask = data["mono_contour_mask"][i].numpy()
    axes[i][6].imshow(mono_contour_mask[0], cmap="gray")
    axes[i][6].set_title("mono contour")

    inflamm_centroid_mask = data["inflamm_centroid_mask"][i].numpy()
    axes[i][7].imshow(inflamm_centroid_mask[0], cmap="gray")
    axes[i][7].set_title("Inflamm centroid")

    lymph_centroid_mask = data["lymph_centroid_mask"][i].numpy()
    axes[i][8].imshow(lymph_centroid_mask[0], cmap="gray")
    axes[i][8].set_title("lymph centroid")

    mono_centroid_mask = data["mono_centroid_mask"][i].numpy()
    axes[i][9].imshow(mono_centroid_mask[0], cmap="gray")
    axes[i][9].set_title("mono centroid")

    # inflamm_weight_maks = data['inflamm_weight_mask'][i].numpy()
    # axes[i][10].imshow(inflamm_weight_maks[0], cmap="jet")
    # axes[i][10].set_title("inflamm weight mask")

    # lymph_weight_mask = data['lymph_weight_mask'][i].numpy()
    # axes[i][11].imshow(lymph_weight_mask[0], cmap="jet")
    # axes[i][11].set_title("lymph weight mask")

    # mono_weight_mask = data['mono_weight_mask'][i].numpy()
    # axes[i][12].imshow(mono_weight_mask[0], cmap="jet")
    # axes[i][12].set_title("mono weight mask")

    inflamm_grad_x = data["inflamm_grad_x"][i].numpy()
    inflamm_grad_y = data["inflamm_grad_y"][i].numpy()
    axes[i][10].imshow(inflamm_grad_x[0], cmap="jet")
    axes[i][10].set_title("inflamm grad x")
    
    lymph_grad_x = data["lymph_grad_x"][i].numpy()
    lymph_grad_y = data["lymph_grad_y"][i].numpy()
    axes[i][11].imshow(inflamm_grad_y[0], cmap="jet")
    axes[i][11].set_title("inflamm grad y")

    mono_grad_x = data["mono_grad_x"][i].numpy()
    mono_grad_y = data["mono_grad_y"][i].numpy()
    axes[i][12].imshow(mono_grad_y[0], cmap="jet")
    axes[i][12].set_title("mono grad y")


    for ax in axes.ravel():
        ax.axis("off")

plt.show()

Visualize Segmentation dataset (NuClick Masks)

In [ ]:
%reset -f
import sys

sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import os
import random
from monkey.data.dataset import get_detection_dataloaders
from monkey.config import TrainingIOConfig
from monkey.data.data_utils import imagenet_denormalise


IOconfig = TrainingIOConfig(
    dataset_dir="/Users/wildsquirrel/Documents/Monkey/patches_256",
    save_dir="./",
)

IOconfig.set_mask_dir(
    "/Users/wildsquirrel/Documents/Monkey/patches_256/annotations/nuclick_masks_processed"
)

batch_size = 8
train_loader, val_loader = get_detection_dataloaders(
    IOconfig,
    val_fold=1,
    dataset_name="segmentation",
    batch_size=batch_size,
    do_augmentation=True,
    augmentation_prob=0.5,
    strong_augmentation=True,
)

In [ ]:
data = next(iter(val_loader))

print(data["id"])
fig, axes = plt.subplots(batch_size, 5, figsize=(12, 40))

for i in range(batch_size):
    image = data["image"][i].numpy()
    image = np.moveaxis(image, 0, 2)
    image = imagenet_denormalise(image)
    axes[i][0].imshow(image)

    inflamm_mask = data["binary_mask"][i].numpy()
    axes[i][1].imshow(inflamm_mask[0], cmap="gray")
    # axes[i][1].imshow(image, alpha=0.3)
    axes[i][1].set_title("binary mask")

    class_mask = data["class_mask"][i].numpy()
    axes[i][2].imshow(class_mask[0], cmap="gray")
    # axes[i][2].imshow(image, alpha=0.3)
    axes[i][2].set_title("lymph")

    axes[i][3].imshow(class_mask[1], cmap="gray")
    # axes[i][3].imshow(image, alpha=0.3)
    axes[i][3].set_title("mono")

    contour_mask = data["contour_mask"][i].numpy()
    axes[i][4].imshow(contour_mask[0], cmap="gray")
    axes[i][4].set_title("contour")

    for ax in axes.ravel():
        ax.axis("off")

plt.show()

In [ ]:
%reset -f
import sys

sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import os
import random
from monkey.data.dataset import get_detection_dataloaders
from monkey.config import TrainingIOConfig
from monkey.data.data_utils import imagenet_denormalise


IOconfig = TrainingIOConfig(
    dataset_dir="/Users/wildsquirrel/Documents/Monkey/patches_256",
    save_dir="./",
)

IOconfig.set_mask_dir(
    "/Users/wildsquirrel/Documents/Monkey/patches_256/annotations/nuclick_masks_processed_v2"
)

batch_size = 8
train_loader, val_loader = get_detection_dataloaders(
    IOconfig,
    val_fold=1,
    dataset_name="segmentation",
    batch_size=batch_size,
    do_augmentation=True,
    augmentation_prob=0.5,
    strong_augmentation=True,
    version=2,
)

In [ ]:
data = next(iter(train_loader))

print(data["id"])
fig, axes = plt.subplots(batch_size, 7, figsize=(12, 40))

for i in range(batch_size):
    image = data["image"][i].numpy()
    image = np.moveaxis(image, 0, 2)
    image = imagenet_denormalise(image)
    axes[i][0].imshow(image)

    inflamm_mask = data["inflamm_mask"][i].numpy()
    axes[i][1].imshow(inflamm_mask[0], cmap="gray")
    # axes[i][1].imshow(image, alpha=0.3)
    axes[i][1].set_title("inflamm mask")

    lymph_mask = data["lymph_mask"][i].numpy()
    axes[i][2].imshow(lymph_mask[0], cmap="gray")
    # axes[i][2].imshow(image, alpha=0.3)
    axes[i][2].set_title("lymph")

    mono_mask = data["mono_mask"][i].numpy()
    axes[i][3].imshow(mono_mask[0], cmap="gray")
    # axes[i][3].imshow(image, alpha=0.3)
    axes[i][3].set_title("mono")

    inflamm_contour_mask = data["inflamm_contour_mask"][i].numpy()
    axes[i][4].imshow(inflamm_contour_mask[0], cmap="gray")
    axes[i][4].set_title("contour")

    lymph_contour_mask = data["lymph_contour_mask"][i].numpy()
    axes[i][5].imshow(lymph_contour_mask[0], cmap="gray")
    axes[i][5].set_title("lymph contour")

    mono_contour_mask = data["mono_contour_mask"][i].numpy()
    axes[i][6].imshow(mono_contour_mask[0], cmap="gray")
    axes[i][6].set_title("mono contour")

    for ax in axes.ravel():
        ax.axis("off")

plt.show()

Visualize additional lymphocyte masks

In [1]:
import sys

sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import re
import cv2
import json
from pprint import pprint
import random

In [ ]:
img_path = "/home/u1910100/Documents/Monkey/additional_patches/images/TCGA-EW-A1PB-DX1_xmin57214_ymin25940_MPP-0.2500.pnpy.npy"
mask_path = "/home/u1910100/Documents/Monkey/additional_patches/masks/TCGA-EW-A1PB-DX1_xmin57214_ymin25940_MPP-0.2500.pnpy.npy"


image_patch = np.load(img_path)
cell_mask = np.load(mask_path)

plt.imshow(cell_mask)
plt.show()